# Inter Annotation Aggreement & Model Evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Tilburg DSS/Thesis/Data

In [ ]:
import pandas as pd

In [ ]:
#import the file including the labels given by the raters
df=pd.read_excel("Manual_Labelling_Annotators_.xlsx")

In [ ]:
df=pd.DataFrame(data=df, columns=['Id', "Tweet", "Annotator1", "Annotator2", "Annotator3"])

#### Sum the number of 0's, -1's and 1's on each row

In [ ]:
df2=(df == 0).astype(int).sum(axis=1)

In [ ]:
df3=(df == -1).astype(int).sum(axis=1)

In [ ]:
df4=(df == 1).astype(int).sum(axis=1)

In [ ]:
frames2=[df,df3,df2,df4]
df5=pd.concat(frames2, axis=1)

In [ ]:
df5.columns =['Id','Tweet', 'Annotator1', 'Annotator2', 'Annotator3',"Negative","Neutral","Positive"]

In [ ]:
df5.to_pickle("Annotation.pkl")

In [ ]:
df_annotator=df5[["Negative","Neutral","Positive"]]

In [ ]:
data=df5[['Annotator1', 'Annotator2', 'Annotator3']]

In [ ]:
import numpy as np

In [ ]:
data2=data.to_numpy()

In [ ]:
transpose = np.array(data).transpose()

## Compute the krippendorff's alpha

In [ ]:
pip install krippendorff

In [ ]:
import krippendorff as kd
kd.alpha(transpose)

In [ ]:
#import the file which includes the annotator's final rate and the BERT models on pre-processed data
df_Models=pd.read_excel("Annotator_models_processedBERT11.xlsx")

In [ ]:
df_Models=df_Models[['id', 'tweet', 'Annotator1', 'Annotator2', 'Annotator3','Polarity',
       'TextBlob_Polarity_unproc',
       'TextBlob_Polarity_lem.1','unprocessed_VADER','VADER_lem_comp', 'BERTweet',
       'Roberta_Latest']]

In [ ]:
#import the file including BERT models on unprocessed data
df_BERT_unproc=pd.read_pickle("BERTweet_unproc.pkl")
df_RoBERTa_unproc=pd.read_pickle("Roberta_Latest_unproc.pkl")

In [ ]:
#take the same sample with the annotated tweets
df_sampled_=df_BERT_unproc.sample(n=100, random_state=1)
df_sampled_roberta=df_RoBERTa_unproc.sample(n=100, random_state=1)

In [ ]:
from functools import reduce  

In [ ]:
# combine the models RoBERTa and BERTweet
#reference: https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns
data_merge = reduce(lambda left, right:  
                     pd.merge(left , right,
                              on = ['id', 'tweet']),
                     [df_sampled_,df_sampled_roberta])
Models1=pd.DataFrame(data_merge)   

In [ ]:
Models1=Models1[['id', 'tweet', 'BERTweet_unproc','Roberta_Latest_unproc']]

In [ ]:
preds=[]
for i in Models1["BERTweet_unproc"]:
        if i["label"].startswith('POS'):
            preds.append(1)
        elif i["label"].startswith('NEU'):
            preds.append(0)
        else:
            preds.append(-1)

In [ ]:
Models1["BERTweet_unproc"]=pd.DataFrame(data=preds)

In [ ]:
preds2=[]
for i in Models1.Roberta_Latest_unproc:
    if i["label"].startswith('p'):
          preds2.append(1)
    elif i["label"].startswith('neu'):
          preds2.append(0)
    else:
          preds2.append(-1)

In [ ]:
Models1["Roberta_Latest_unproc"]=pd.DataFrame(data=preds2)

In [ ]:
data_merge1 = pd.concat([df_Models, Models1], axis=1)


In [ ]:
#polarity: ground truth labels 
data_merge1=data_merge1[['id', 'tweet', 'Annotator1', 'Annotator2', 'Annotator3', 'Polarity',
       'TextBlob_Polarity_unproc', 'TextBlob_Polarity_lem.1',
       'unprocessed_VADER', 'VADER_lem_comp', 'BERTweet', 'Roberta_Latest',
       'BERTweet_unproc', 'Roberta_Latest_unproc']]

In [ ]:
data_merge1.to_pickle("Manual_Labelling_Annotators_Models.pkl")

In [ ]:
df_Models2=data_merge1

## Evaluation Metrics Calculation

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Show the classification report
print(classification_report(df_Models2['Polarity'], df_Models2['Roberta_Latest']))

In [ ]:
# Show the classification report
print(classification_report(df_Models2['Polarity'], df_Models2['Roberta_Latest_unproc']))

In [ ]:
print(classification_report(df_Models2['Polarity'], df_Models2['BERTweet_unproc']))

In [ ]:
print(classification_report(df_Models2['Polarity'], df_Models2['BERTweet']))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report


print(classification_report(df_Models2['Polarity'], df_Models2['TextBlob_Polarity_lem.1']))

In [ ]:
print(classification_report(df_Models2['Polarity'], df_Models2['TextBlob_Polarity_unproc']))

In [ ]:
# Show the classification report
print(classification_report(df_Models2['Polarity'], df_Models2['unprocessed_VADER']))

In [ ]:
print(classification_report(df_Models2['Polarity'], df_Models2['VADER_lem_comp']))